In [1]:
import pandas as pd
import numpy as np
import sqlite3
from dotenv import load_dotenv
import os
import psycopg2
from psycopg2 import extras
load_dotenv();

### Load Data

In [10]:
campaign = pd.read_csv("../data/Campaign.csv", encoding="latin-1")
profile_creation = pd.read_csv("../data/EPO_Teradata_Employer_Profile_Creation_Report.csv")
job_seeker = pd.read_csv("../data/EPO_Teradata_Job Seeker_Profile_Creation_Report.csv")
job_board = pd.read_csv("../data/EPO_Teradata_Job_Board_Sales_Report.csv")
feedback = pd.read_csv("../data/Feedback__c.csv", encoding="latin-1", low_memory=False)
account = pd.read_csv("../data/SalesForce_Account.csv", encoding="latin-1", low_memory=False)
sf_case = pd.read_csv("../data/SalesForce_Case.csv", encoding="latin-1", low_memory=False)
info_c = pd.read_csv("../data/SalesForce_Hire_Information__c.csv", encoding="latin-1")
opp = pd.read_csv("../data/SalesForce_Opportunity.csv", low_memory=False)
record = pd.read_csv("../data/SalesForce_RecordType.csv")

#no columns to pull data from
#sales_2018 = pd.read_csv("../data/SalesForce_2018Activities.csv")
#sales_force = pd.read_csv("../data/SalesForce_Contact.csv")
#email = pd.read_csv("../data/vr__VR_Email_History_Contact__c.csv")

ls = [campaign, profile_creation, job_seeker, job_board, feedback, account, sf_case, 
      info_c, opp, record]
ls_name = ["campaign", "profile_creation", "job_seeker", "job_board", "feedback", "account", "sf_case", 
      "info_c", "opp", "record"]

# Lower and replace spaces with "_" in column names
for x in ls:
    x.columns = map(str.lower, x.columns.str.replace(" ", "_").str.replace("?", ""))

df_dict = dict(zip(ls_name, ls))
print("---Dataframes Shapes---")
for x in range(len(df_dict)):
    print(f"{list(df_dict.keys())[x]} Shape: {list(df_dict.values())[x].shape}")  

---Dataframes Shapes---
campaign Shape: (1386, 103)
profile_creation Shape: (778, 5)
job_seeker Shape: (4342, 5)
job_board Shape: (521, 8)
feedback Shape: (15807, 70)
account Shape: (16858, 168)
sf_case Shape: (14845, 56)
info_c Shape: (30754, 34)
opp Shape: (10849, 130)
record Shape: (80, 13)


In [3]:
def df_sql(df_ls,name, conn):
    """
    Load all data into a sqlite3 DB
    """
    
    conn = conn
    
    for x in range(len(df_ls)):
        df_ls[x].to_sql(f"{name[x]}", conn, index=False)
        
    return   

def create_table(conn):
    """
    Returns a list a create table statements as strings
    """
    
    curs = conn.cursor()

    query = curs.execute("SELECT * FROM sqlite_master WHERE type='table'").fetchall()
    table_ls = []

    for x in range(len(query)):
        table_ls.append(query[x][4].replace("\n", "") + ";")
    
    return table_ls

def insert_pg(conn, name):
    """
    Append insert statements to a list
    """
    
    curs = conn.cursor()
    insert_ls = []
    
    for x in range(len(name)):
        curs.execute(f"SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='{name[x]}';")
        insert_str = f"INSERT INTO {name[x]} ("
        for y in curs:
            col_ls = []
            col_ls.append(y[0])
            for z in range(len(col_ls)):
                insert_str += col_ls[z] + "," + " "
        insert_str = insert_str[:-2] # remove the "," and " " from the last line
        insert_str += ") VALUES %s"
        insert_ls.append(insert_str)
        
    return insert_ls

### SQLite3

In [5]:
# Make connection
conn = sqlite3.Connection("../data/heroes.sqlite3")

# Load the DB
df_sql(ls, ls_name, conn)

# Extract CREATE TABLE statements and append to list
table_ls = create_table(conn)

### Psycopg2 and Azure

In [6]:
# Load credentials from .env
name = os.getenv("AZURE_DB_NAME")
pw = os.getenv("AZURE_PASS")
host = os.getenv("AZURE_HOST")
user = os.getenv("AZURE_USER")
ssl = os.getenv("AZURE_SSLMODE")

# Connect
conn_string = f"host={host} user={user} dbname={name} password={pw} sslmode={ssl}"
pg_conn = psycopg2.connect(conn_string)
pg_curs = pg_conn.cursor()

# Drop and Create tables
for x in ls_name:
    pg_curs.execute(f"DROP TABLE IF EXISTS {x};")
    pg_conn.commit()

for x in range(len(table_ls)):
    pg_curs.execute(table_ls[x])
    pg_conn.commit()

# Gather insert statements to a list
insert_ls = insert_pg(pg_conn, ls_name)

# Insert the Data
for x in range(len(ls_name)):
    curs = conn.cursor()
    data = curs.execute(f"SELECT * FROM {ls_name[x]}").fetchall()
    query = insert_ls[x]
    extras.execute_values(pg_curs, query, data)
    pg_conn.commit
    print(f"{ls_name[x]} data inserted")
    
curs.close()
conn.close()
pg_curs.close()
pg_conn.close()

campaign data inserted
profile_creation data inserted
job_seeker data inserted
job_board data inserted
feedback data inserted
account data inserted
sf_case data inserted
info_c data inserted
opp data inserted
record data inserted
